_We're going to be using pandas an json, so lets start with that._

In [34]:
import json
import pandas as pd
import os
from pandas.io.json import json_normalize
import geopandas

# Looking at the json files

_Before we do any real work, can we list the files from our target directory?_

In [35]:
!pwd
! ls ../APIs

/home/vagrant/git/BigData/democratizing_weather_data/democratizing_weather_data/streaming/prototypes
2yahoo.json	     wsdot.TrafficFlows.json  yahoo_weather1line.json
API_index.csv	     wsdot.Traveltimes.json   yahoo_weather2.json
openweathermap.json  yahoo_weather	      yahoo_weather.json


_Nice! Let's examine a json file_

In [36]:
!head ../APIs/yahoo_weather.json

{
    "query": {
        "count": 1,
        "created": "2017-07-29T23:51:25Z",
        "lang": "en-US",
        "diagnostics": {
            "publiclyCallable": "true",
            "url": {
                "execution-start-time": "1",
                "execution-stop-time": "5",


# Exploring JSON from the Yahoo Weather API

_Read a json file from disk (we'll use ../API/<tbd>.json
We should reall use the yahoo api output, since we know how to convert that to a csv_

Source [Andy Hayden on ghthub.io](http://hayd.github.io/2013/pandas-json)

In [37]:
df = pd.read_json('../APIs/yahoo_weather.json')
df

,query
count,1
created,2017-07-29T23:51:25Z
diagnostics,"{'publiclyCallable': 'true', 'url': {'executio..."
lang,en-US
results,"{'channel': {'units': {'distance': 'mi', 'pres..."


_Not much of a dataframe. Let's re-read and nomralize, drilling into the results section._

Read file with json Library. Source: [Scott Robinson's Stack Abuse Article ](http://stackabuse.com/reading-and-writing-json-to-a-file-in-python/)

In [38]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_df = json_normalize(raw_json['query']) 
yahoo_weather_df

,count,created,diagnostics.build-version,diagnostics.publiclyCallable,diagnostics.service-time,diagnostics.url.content,diagnostics.url.execution-start-time,diagnostics.url.execution-stop-time,diagnostics.url.execution-time,diagnostics.user-time,...,results.channel.location.region,results.channel.title,results.channel.ttl,results.channel.units.distance,results.channel.units.pressure,results.channel.units.speed,results.channel.units.temperature,results.channel.wind.chill,results.channel.wind.direction,results.channel.wind.speed
0,1,2017-07-29T23:51:25Z,2.0.164,true,4,http://weather-ydn-yql.media.yahoo.com:4080/v3...,1,5,4,6,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


_One row, 50 columns. Let's look at data types._

In [39]:
yahoo_weather_df.dtypes

count                                     int64
created                                  object
diagnostics.build-version                object
diagnostics.publiclyCallable             object
diagnostics.service-time                 object
diagnostics.url.content                  object
diagnostics.url.execution-start-time     object
diagnostics.url.execution-stop-time      object
diagnostics.url.execution-time           object
diagnostics.user-time                    object
lang                                     object
results.channel.astronomy.sunrise        object
results.channel.astronomy.sunset         object
results.channel.atmosphere.humidity      object
results.channel.atmosphere.pressure      object
results.channel.atmosphere.rising        object
results.channel.atmosphere.visibility    object
results.channel.description              object
results.channel.image.height             object
results.channel.image.link               object
results.channel.image.title             

_That's a lot of data, all of type object. For the Yahoo Weather API, most of the good stuff lives in_ results.channel
_Let's dig deeper_

In [40]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_df = json_normalize(raw_json['query']['results']['channel']) 
yahoo_weather_df

,astronomy.sunrise,astronomy.sunset,atmosphere.humidity,atmosphere.pressure,atmosphere.rising,atmosphere.visibility,description,image.height,image.link,image.title,...,location.region,title,ttl,units.distance,units.pressure,units.speed,units.temperature,wind.chill,wind.direction,wind.speed
0,6:10 am,8:19 pm,58,1006.0,0,16.1,"Yahoo! Weather for Sunnyvale, CA, US",18,http://weather.yahoo.com,Yahoo! Weather,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


_Looking more like a dataframe.  Let's look at fields & datatypes_

In [41]:
yahoo_weather_df.dtypes

astronomy.sunrise        object
astronomy.sunset         object
atmosphere.humidity      object
atmosphere.pressure      object
atmosphere.rising        object
atmosphere.visibility    object
description              object
image.height             object
image.link               object
image.title              object
image.url                object
image.width              object
item.condition.code      object
item.condition.date      object
item.condition.temp      object
item.condition.text      object
item.description         object
item.forecast            object
item.guid.isPermaLink    object
item.lat                 object
item.link                object
item.long                object
item.pubDate             object
item.title               object
language                 object
lastBuildDate            object
link                     object
location.city            object
location.country         object
location.region          object
title                    object
ttl     

_Everything's an object.  If we want to do any real analysis, we're going to need to do some type conversion. Also note, that the field names have a hierarchy to them.  So maybe we can drill deeper into just the fields we want_

_Drill Down: atmosphere_

In [42]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_atmosphere_df = json_normalize(raw_json['query']['results']['channel']['atmosphere']) 
yahoo_weather_atmosphere_df

,humidity,pressure,rising,visibility
0,58,1006.0,0,16.1


_Drill Down: image_

In [43]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_image_df = json_normalize(raw_json['query']['results']['channel']['image']) 
yahoo_weather_image_df

,height,link,title,url,width
0,18,http://weather.yahoo.com,Yahoo! Weather,http://l.yimg.com/a/i/brand/purplelogo//uh/us/...,142


_Drill Down: item_

In [44]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_item_df = json_normalize(raw_json['query']['results']['channel']['item']) 
yahoo_weather_item_df

,condition.code,condition.date,condition.temp,condition.text,description,forecast,guid.isPermaLink,lat,link,long,pubDate,title
0,32,"Sat, 29 Jul 2017 04:00 PM PDT",75,Sunny,"<![CDATA[<img src=""http://l.yimg.com/a/i/us/we...","[{'code': '34', 'date': '29 Jul 2017', 'day': ...",false,37.371609,http://us.rd.yahoo.com/dailynews/rss/weather/C...,-122.038254,"Sat, 29 Jul 2017 04:00 PM PDT","Conditions for Sunnyvale, CA, US at 04:00 PM PDT"


_Drill Down: item.forecast_

In [45]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_forecast_df = json_normalize(raw_json['query']['results']['channel']['item']['forecast']) 
yahoo_weather_forecast_df

,code,date,day,high,low,text
0,34,29 Jul 2017,Sat,78,58,Mostly Sunny
1,32,30 Jul 2017,Sun,84,60,Sunny
2,32,31 Jul 2017,Mon,87,61,Sunny
3,32,01 Aug 2017,Tue,88,64,Sunny
4,34,02 Aug 2017,Wed,90,66,Mostly Sunny
5,30,03 Aug 2017,Thu,88,68,Partly Cloudy
6,34,04 Aug 2017,Fri,88,67,Mostly Sunny
7,32,05 Aug 2017,Sat,86,66,Sunny
8,34,06 Aug 2017,Sun,83,63,Mostly Sunny
9,34,07 Aug 2017,Mon,79,61,Mostly Sunny


_Drill Down:condition_

In [46]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_conditions_df = json_normalize(raw_json['query']['results']['channel']['item']['condition']) 
yahoo_weather_conditions_df

,code,date,temp,text
0,32,"Sat, 29 Jul 2017 04:00 PM PDT",75,Sunny


In [47]:
yahoo_weather_conditions_df.dtypes

code    object
date    object
temp    object
text    object
dtype: object

_Drill Down: wind_

In [48]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_atmosphere_df = json_normalize(raw_json['query']['results']['channel']['wind']) 
yahoo_weather_atmosphere_df

,chill,direction,speed
0,75,355,18


# Exploring JSON from the Open Weather Map API

In [49]:
!pwd
! ls ../APIs
! cat ../APIs/openweathermap.json

/home/vagrant/git/BigData/democratizing_weather_data/democratizing_weather_data/streaming/prototypes
2yahoo.json	     wsdot.TrafficFlows.json  yahoo_weather1line.json
API_index.csv	     wsdot.Traveltimes.json   yahoo_weather2.json
openweathermap.json  yahoo_weather	      yahoo_weather.json
{
    "coord": {
        "lon": 139.01,
        "lat": 35.02
    },
    "weather": [
        {
            "id": 800,
            "main": "Clear",
            "description": "clear sky",
            "icon": "01n"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 285.514,
        "pressure": 1013.75,
        "humidity": 100,
        "temp_min": 285.514,
        "temp_max": 285.514,
        "sea_level": 1023.22,
        "grnd_level": 1013.75
    },
    "wind": {
        "speed": 5.52,
        "deg": 311
    },
    "clouds": {
        "all": 0
    },
    "dt": 1485792967,
    "sys": {
        "message": 0.0025,
        "country": "JP",
        "sunrise": 1485726240,
        "sunset"

In [50]:
with open('../APIs/openweathermap.json') as json_file:
  raw_json = json.load(json_file)

openweathermap_df = json_normalize(raw_json) 
openweathermap_df

,base,clouds.all,cod,coord.lat,coord.lon,dt,id,main.grnd_level,main.humidity,main.pressure,...,main.temp_max,main.temp_min,name,sys.country,sys.message,sys.sunrise,sys.sunset,weather,wind.deg,wind.speed
0,stations,0,200,35.02,139.01,1485792967,1907296,1013.75,100,1013.75,...,285.514,285.514,Tawarano,JP,0.0025,1485726240,1485763863,"[{'id': 800, 'main': 'Clear', 'description': '...",311,5.52


In [51]:
openweathermap_df.dtypes

base                object
clouds.all           int64
cod                  int64
coord.lat          float64
coord.lon          float64
dt                   int64
id                   int64
main.grnd_level    float64
main.humidity        int64
main.pressure      float64
main.sea_level     float64
main.temp          float64
main.temp_max      float64
main.temp_min      float64
name                object
sys.country         object
sys.message        float64
sys.sunrise          int64
sys.sunset           int64
weather             object
wind.deg             int64
wind.speed         float64
dtype: object

_May of the data types are already numeric.  Pretty cool. Let's dive into the weather object._

In [52]:
with open('../APIs/openweathermap.json') as json_file:
  raw_json = json.load(json_file)

openweathermap_weather_df = json_normalize(raw_json['weather']) 
openweathermap_weather_df

,description,icon,id,main
0,clear sky,01n,800,Clear


# One folder, one dataframe from multiple json files

The yahoo\_weather sufolder contains two json files from the Yahoo weather API.  We've moved all the linefeeds, but that won't affect our ability to make dataframes

In [53]:
! ls -1 ../APIs/yahoo_weather

yahoo_weather1line.json
yahoo_weather2.json


_Uncomment the lines below to see the contets of the files_

In [54]:
#! head ../APIs/yahoo_weather/yahoo_weather1line.json

#! echo ""
#! echo "===== NEXT FILE ==="
#! echo ""

#! cat ../APIs/yahoo_weather/yahoo_weather2.json

In [55]:
top_folder = "../APIs/yahoo_weather"
first_time = True

for filename in os.listdir(top_folder):
    filepath = top_folder + "/" + filename

    with open(filepath) as json_file:
      raw_json = json.load(json_file)

    new_json = json_normalize(raw_json['query']['results']['channel']['atmosphere'])
    
    if (first_time):
        first_time = False
        multifile_df = new_json
    else:
        multifile_df = multifile_df.append(new_json, ignore_index=True)
    
multifile_df

,humidity,pressure,rising,visibility
0,44,1019.0,0,16.1
1,58,1006.0,0,16.1


In [136]:
# !ls ../../../../../../from_bitnami/json
# !head ../../../../../../from_bitnami/json/2017-08-13-22-38-57
df = pd.read_json('../../../../../../from_bitnami/json/2017-08-13-22-38-57')
df
df.dtypes

BarometricPressure         float64
Latitude                   float64
Longitude                  float64
PrecipitationInInches      float64
ReadingTime                 object
RelativeHumidity           float64
SkyCoverage                 object
StationID                    int64
StationName                 object
TemperatureInFahrenheit    float64
Visibility                 float64
WindDirection              float64
WindDirectionCardinal       object
WindGustSpeedInMPH         float64
WindSpeedInMPH             float64
dtype: object

In [144]:
top_folder = "../../../../../../from_bitnami/json"
first_time = True

for filename in os.listdir(top_folder):
    filepath = top_folder + "/" + filename

    with open(filepath) as json_file:
      raw_json = json.load(json_file)

    #new_json = json_normalize(raw_json['query']['results']['channel']['atmosphere'])
    new_json = json_normalize(raw_json)
    
    if (first_time):
        first_time = False
        multifile_df = new_json
    else:
        multifile_df = multifile_df.append(new_json, ignore_index=True)
    
multifile_df

,AverageTime,BarometricPressure,CurrentTime,Description,Distance,EndPoint.Description,EndPoint.Direction,EndPoint.Latitude,EndPoint.Longitude,EndPoint.MilePost,...,StationID,StationName,TemperatureInFahrenheit,TimeUpdated,TravelTimeID,Visibility,WindDirection,WindDirectionCardinal,WindGustSpeedInMPH,WindSpeedInMPH
0,31.0,NaN,33.0,Everett to Downtown Seattle using HOV lanes,26.72,I-5 @ University St in Seattle,S,47.609294,-122.331759,165.83,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,2.0,NaN,NaN,NaN,NaN,NaN
1,28.0,NaN,27.0,Downtown Seattle to Everett using HOV lanes,26.94,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,3.0,NaN,NaN,NaN,NaN,NaN
2,28.0,NaN,27.0,Downtown Seattle to Everett,26.94,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,4.0,NaN,NaN,NaN,NaN,NaN
3,11.0,NaN,11.0,Downtown Bellevue to Issaquah,9.55,I-90 @ Front St in Issaquah,E,47.541799,-122.037396,16.96,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,5.0,NaN,NaN,NaN,NaN,NaN
4,11.0,NaN,11.0,Downtown Bellevue to Issaquah using HOV lanes,9.55,I-90 @ Front St in Issaquah,E,47.541799,-122.037396,16.96,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,6.0,NaN,NaN,NaN,NaN,NaN
5,10.0,NaN,10.0,Issaquah to Downtown Bellevue using HOV lanes,9.48,I-405 @ NE 8th St in Bellevue,N,47.613610,-122.187970,13.60,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,7.0,NaN,NaN,NaN,NaN,NaN
6,12.0,NaN,12.0,Issaquah to Downtown Bellevue,9.48,I-405 @ NE 8th St in Bellevue,N,47.613610,-122.187970,13.60,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,8.0,NaN,NaN,NaN,NaN,NaN
7,26.0,NaN,26.0,Downtown Bellevue to Everett,26.06,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,9.0,NaN,NaN,NaN,NaN,NaN
8,27.0,NaN,28.0,Everett to Downtown Bellevue,26.34,I-405 @ NE 8th St in Bellevue,S,47.613800,-122.188920,13.33,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,10.0,NaN,NaN,NaN,NaN,NaN
9,26.0,NaN,26.0,Downtown Bellevue to Everett using HOV lanes,26.06,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,...,NaN,NaN,NaN,/Date(1502665800000-0700)/,11.0,NaN,NaN,NaN,NaN,NaN


In [56]:
multifile_df.dtypes

humidity      object
pressure      object
rising        object
visibility    object
dtype: object

# One file, multiple json objects

_File 2yahoo.json contains two json objects from the yahoo weather api.  Each object has had newlines removed, so it only occupies one line of text. Uncomment the line below to have a look._

In [57]:
# !cat ../APIs/2yahoo.json

_The linux word count (wc) with the line (-l) option can count the lines int he file)_

In [58]:
!wc -l < ../APIs/2yahoo.json

2


_Read the json file into a dataframe ... what do we get?_

In [59]:
df3 = pd.read_json('../APIs/2yahoo.json', lines=True)
df3

,query
0,"{'count': 1, 'created': '2017-07-29T23:51:25Z'..."
1,"{'count': 1, 'created': '2017-08-01T02:14:07Z'..."


_... not much.  Nice that we got a mulit-row dataframe, but this json needs to be normalized to be useful_

Code is pretty similar to the multi-file case, just using json.loads() instead of json.load()
Source: [Stack Overflow](https://stackoverflow.com/questions/8009882/how-to-read-large-file-line-by-line-in-python)

In [60]:
first_time = True

with open('../APIs/2yahoo.json') as f_one_json_per_line:
    for line in f_one_json_per_line:
        raw_json = json.loads(line)
        new_json = json_normalize(raw_json['query']['results']['channel']['atmosphere'])
        
        if (first_time):
            first_time = False
            multiline_df = new_json
        else:
            multiline_df = multiline_df.append(new_json, ignore_index=True)
        
multiline_df

,humidity,pressure,rising,visibility
0,58,1006.0,0,16.1
1,44,1019.0,0,16.1


# WSDOT Exploring Weather Data

In [61]:
# wsdot_Weather_df = pd.read_json('../APIs/wsdot.Traveltimes.json')
# wsdot_TravelTimes_df

# WSDOT Exploring Transportation Data

_Begin by loading the WSDOT Transportaiton Traveltims json file_

In [66]:
wsdot_TravelTimes_df = pd.read_json('../APIs/wsdot.Traveltimes.json')
wsdot_TravelTimes_df

,AverageTime,CurrentTime,Description,Distance,EndPoint,Name,StartPoint,TimeUpdated,TravelTimeID
0,30,33,Everett to Downtown Seattle using HOV lanes,26.72,{'Description': 'I-5 @ University St in Seattl...,HOV Everett-Seattle (SB AM),"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1500575100000-0700)/,2
1,30,29,Downtown Seattle to Everett using HOV lanes,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",HOV Seattle-Everett (NB PM),{'Description': 'I-5 @ University St in Seattl...,/Date(1500575100000-0700)/,3
2,30,29,Downtown Seattle to Everett,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett (NB PM),{'Description': 'I-5 @ University St in Seattl...,/Date(1500575100000-0700)/,4
3,11,11,Downtown Bellevue to Issaquah,9.55,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah (EB PM),{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1500575100000-0700)/,5
4,11,11,Downtown Bellevue to Issaquah using HOV lanes,9.55,"{'Description': 'I-90 @ Front St in Issaquah',...",HOV Bellevue-Issaquah (EB PM),{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1500575100000-0700)/,6
5,10,11,Issaquah to Downtown Bellevue using HOV lanes,9.48,{'Description': 'I-405 @ NE 8th St in Bellevue...,HOV Issaquah-Bellevue (WB AM),"{'Description': 'I-90 @ Front St in Issaquah',...",/Date(1500575100000-0700)/,7
6,12,12,Issaquah to Downtown Bellevue,9.48,{'Description': 'I-405 @ NE 8th St in Bellevue...,Issaquah-Bellevue (WB AM),"{'Description': 'I-90 @ Front St in Issaquah',...",/Date(1500575100000-0700)/,8
7,28,27,Downtown Bellevue to Everett,26.06,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Bellevue-Everett (NB PM),{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1500575100000-0700)/,9
8,30,30,Everett to Downtown Bellevue,26.34,{'Description': 'I-405 @ NE 8th St in Bellevue...,Everett-Bellevue (SB AM),"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1500575100000-0700)/,10
9,26,27,Downtown Bellevue to Everett using HOV lanes,26.06,"{'Description': 'I-5 @ 41st St in Everett', 'D...",HOV Bellevue-Everett (NB PM),{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1500575100000-0700)/,11


In [67]:
wsdot_TravelTimes_df.dtypes

AverageTime       int64
CurrentTime       int64
Description      object
Distance        float64
EndPoint         object
Name             object
StartPoint       object
TimeUpdated      object
TravelTimeID      int64
dtype: object

_Fairly flat structure; some of the top-lvel types are numeric already. Let's try to crack open the STartpoint and Endpoint, so we can get some lat/long data._  *Note: this code will raise an error

In [68]:
with open('../APIs/wsdot.Traveltimes.json') as json_file:
  raw_json = json.load(json_file)

wsdot_TravelTimes_df = json_normalize(raw_json['StartPoint']) 
wsdot_TravelTimes_df

TypeError: list indices must be integers or slices, not str

_Turns out we need to use row-indexing (below), but that only gets us one row at time._ 

In [69]:
with open('../APIs/wsdot.Traveltimes.json') as json_file:
  raw_json = json.load(json_file)

wsdot_TravelTimes_df = json_normalize(raw_json[0]['StartPoint'])
print(wsdot_TravelTimes_df)

                Description Direction  Latitude  Longitude  MilePost RoadName
0  I-5 @ 41st St in Everett         S  47.92428 -122.26548    192.55      I-5


In [70]:
wsdot_TravelTimes_df.dtypes

Description     object
Direction       object
Latitude       float64
Longitude      float64
MilePost       float64
RoadName        object
dtype: object

To normalize everything, we need to ....

1. Load the json file as a list of lines, so we can iterate over it.

2. Also load the json file as a raw json object.

3. Iterate over the lines, appendeing each one to a "master" dataframe

In [71]:
# Load the json as a list, so we can iterate over it.
with open('../APIs/wsdot.Traveltimes.json') as json_file:
  file_data = json_file.read()
    
list_json = json.loads(file_data)


# Load the json file as json, so we can normalize it.
with open('../APIs/wsdot.Traveltimes.json') as json_file:
  raw_json = json.load(json_file)


#iterate over the list_json. for each row, normalize the row_json
ii = 0

for travelTime in list_json:
    travel_df = json_normalize(raw_json[ii]['StartPoint'])
    if (ii == 0):
        wsdot_TravelTimesStartPoint_df= json_normalize(raw_json[ii]['StartPoint'])
    else:
        wsdot_TravelTimesStartPoint_df = wsdot_TravelTimesStartPoint_df.append(travel_df, ignore_index=True)
    ii= ii+ 1
   
wsdot_TravelTimesStartPoint_df

,Description,Direction,Latitude,Longitude,MilePost,RoadName
0,I-5 @ 41st St in Everett,S,47.924280,-122.265480,192.55,I-5
1,I-5 @ University St in Seattle,N,47.609294,-122.331759,165.83,I-5
2,I-5 @ University St in Seattle,N,47.609294,-122.331759,165.83,I-5
3,I-405 @ NE 8th St in Bellevue,S,47.613800,-122.188920,13.33,I-405
4,I-405 @ NE 8th St in Bellevue,S,47.613800,-122.188920,13.33,I-405
5,I-90 @ Front St in Issaquah,W,47.541799,-122.037396,16.96,I-90
6,I-90 @ Front St in Issaquah,W,47.541799,-122.037396,16.96,I-90
7,I-405 @ NE 8th St in Bellevue,N,47.613610,-122.187970,13.60,I-405
8,I-5 @ 41st St in Everett,S,47.924280,-122.265480,192.55,I-5
9,I-405 @ NE 8th St in Bellevue,N,47.613610,-122.187970,13.60,I-405


In [72]:
wsdot_TravelTimesStartPoint_df.describe()

,Latitude,Longitude,MilePost
count,180.000000,180.000000,180.000000
mean,46.301362,-120.312675,88.487111
std,6.102723,15.708814,80.749049
min,0.000000,-122.891484,0.100000
25%,46.672680,-122.508103,13.330000
50%,47.464051,-122.331130,71.025000
75%,47.613610,-122.206552,165.830000
max,48.152323,0.000000,307.900000


In [73]:
# Load the json as a list, so we can iterate over it.
with open('../APIs/wsdot.Traveltimes.json') as json_file:
  file_data = json_file.read()
    
list_json = json.loads(file_data)


# Load the json file as json, so we can normalize it.
with open('../APIs/wsdot.Traveltimes.json') as json_file:
  raw_json = json.load(json_file)


#iterate over the list_json. for each row, normalize the row_json
ii = 0

for travelTime in list_json:
  travel_df = json_normalize(raw_json[ii]['EndPoint'])
  if (ii == 0):
    wsdot_TravelTimesEndPoint_df= json_normalize(raw_json[ii]['EndPoint'])
  else:
    wsdot_TravelTimesEndPoint_df = wsdot_TravelTimesEndPoint_df.append(travel_df, ignore_index=True)
  ii= ii+ 1
   
wsdot_TravelTimesEndPoint_df

,Description,Direction,Latitude,Longitude,MilePost,RoadName
0,I-5 @ University St in Seattle,S,47.609294,-122.331759,165.83,I-5
1,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,I-5
2,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,I-5
3,I-90 @ Front St in Issaquah,E,47.541799,-122.037396,16.96,I-90
4,I-90 @ Front St in Issaquah,E,47.541799,-122.037396,16.96,I-90
5,I-405 @ NE 8th St in Bellevue,N,47.613610,-122.187970,13.60,I-405
6,I-405 @ NE 8th St in Bellevue,N,47.613610,-122.187970,13.60,I-405
7,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,I-5
8,I-405 @ NE 8th St in Bellevue,S,47.613800,-122.188920,13.33,I-405
9,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,I-5


In [74]:
wsdot_TravelTimesEndPoint_df.describe()

,Latitude,Longitude,MilePost
count,180.000000,180.000000,180.000000
mean,47.072796,-122.366538,99.269722
std,0.859379,0.241429,89.226318
min,45.353550,-122.891484,0.110000
25%,47.047885,-122.560915,13.330000
50%,47.464051,-122.331759,116.520000
75%,47.613610,-122.219631,165.830000
max,48.152323,-120.599140,307.350000


# Joining Transportation and Weather with geopands
- Gather up the data frames (one weather, one transporattion, both in same proximity)
- Load the geopandas library
- datafraems to geoframes
- do the join

_We have a travel dataframe from WSDOT. It has "Latitude" and "Longituded" cordinate fields_

In [75]:
wsdot_TravelTimesEndPoint_df.head()

,Description,Direction,Latitude,Longitude,MilePost,RoadName
0,I-5 @ University St in Seattle,S,47.609294,-122.331759,165.83,I-5
1,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,I-5
2,I-5 @ 41st St in Everett,N,47.924280,-122.265480,192.77,I-5
3,I-90 @ Front St in Issaquah,E,47.541799,-122.037396,16.96,I-90
4,I-90 @ Front St in Issaquah,E,47.541799,-122.037396,16.96,I-90


_We also have an OpenWeatherMap dataframe. It has "coord.lat" and "coord.lon"_

In [76]:
with open('../APIs/openweathermap.json') as json_file:
  raw_json = json.load(json_file)

openweathermap_df = json_normalize(raw_json) 
openweathermap_df

,base,clouds.all,cod,coord.lat,coord.lon,dt,id,main.grnd_level,main.humidity,main.pressure,...,main.temp_max,main.temp_min,name,sys.country,sys.message,sys.sunrise,sys.sunset,weather,wind.deg,wind.speed
0,stations,0,200,35.02,139.01,1485792967,1907296,1013.75,100,1013.75,...,285.514,285.514,Tawarano,JP,0.0025,1485726240,1485763863,"[{'id': 800, 'main': 'Clear', 'description': '...",311,5.52


In [85]:
openweathermap_df.dtypes

base                object
clouds.all           int64
cod                  int64
coord.lat          float64
coord.lon          float64
dt                   int64
id                   int64
main.grnd_level    float64
main.humidity        int64
main.pressure      float64
main.sea_level     float64
main.temp          float64
main.temp_max      float64
main.temp_min      float64
name                object
sys.country         object
sys.message        float64
sys.sunrise          int64
sys.sunset           int64
weather             object
wind.deg             int64
wind.speed         float64
dtype: object

In [86]:
openweathermap_df.cod

0    200
Name: cod, dtype: int64

In [88]:
openweathermap_df['coord.lat']
#df['Project.Fwd_Primer']

0    35.02
Name: coord.lat, dtype: float64

In [106]:
d = {"age":25}

_Let's convert both of these into geogrames_
Try this from [stackexchange article Convert a pandas DataFrame to a GeoDataFrame](https://gis.stackexchange.com/questions/174159/convert-a-pandas-dataframe-to-a-geodataframe/174168)



from geopandas import GeoDataFrame

from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(df.Lon, df.Lat)]

df = df.drop(['Lon', 'Lat'], axis=1)

crs = {'init': 'epsg:4326'}

geo_df = GeoDataFrame(df, crs=crs, geometry=geometry)

In [122]:
crs = {'init':'espg:4326'}

In [121]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
from __future__ import braces

geometry = [Point(xy) for xy in zip(openweathermap_df['coord.lon'], openweathermap_df['coord.lat'])
#openweathermap_df = openweathermap_df.drop(['coord.lon'], ['coord.lat'], axis=1)
            
crs = {'init':'espg:4326'}

            #crs = dict([("init","epsg:4326")])     

geo_df = GeoDataFrame(openweathermap_df, crs, geometry)
geo_df

SyntaxError: invalid syntax (<ipython-input-121-ffc6bba22afa>, line 8)

Try this from https://geohackweek.github.io/vector/04-geopandas-intro/

In [99]:
gs = GeoSeries([Point(-120, 45), Point(-121.2, 46), Point(-122.9, 47.5)])
gs

NameError: name 'GeoSeries' is not defined

This shows promise
https://stackoverflow.com/questions/37728540/create-a-geodataframe-from-a-geojson-object
    
In [1]: from geojson import Feature, Point, FeatureCollection

In [2]: my_feature = Feature(geometry=Point((1.6432, -19.123)), properties={"country": "Spain"})

In [3]: my_other_feature = Feature(geometry=Point((-80.234, -22.532)), properties={'country': 'Brazil'})

In [4]: collection = FeatureCollection([my_feature, my_other_feature])


In [6]: import geopandas

In [7]: geopandas.GeoDataFrame.from_features(collection['features'])
Out[7]:
  country                 geometry
0   Spain   POINT (1.6432 -19.123)
1  Brazil  POINT (-80.234 -22.532)

In [145]:
from geojson import Feature, Point, FeatureCollection

In [146]:
my_feature = Feature(geometry=Point((1.6432, -19.123)), properties={"country": "Spain"})

In [147]:
my_other_feature = Feature(geometry=Point((-80.234, -22.532)), properties={'country': 'Brazil'})

In [152]:
# collection = FeatureCollection([my_feature, my_other_feature])
collection = FeatureCollection([my_feature])
collection.add(my_other_feature)

AttributeError: add

In [149]:
import geopandas

In [150]:
geopandas.GeoDataFrame.from_features(collection['features'])

,country,geometry
0,Spain,POINT (1.6432 -19.123)
1,Brazil,POINT (-80.23399999999999 -22.532)


Okay, I think this will work (above). I just have to ....
 - Iterate over my non-geo dataframe.
 - make a feature for each row
 - incrementally add each new feature to a collection
 - make the geopandas data frame

# Future Work
- Figure out how to join and merge stuff (geogson)
- Parse dates from objects into something actionable
- Go back to the tranporation df's, do I really need to load twice?
